# Build your first agent!



# Environment


In [ ]:
!pip install -qU google-adk

In [ ]:
import os
PROJECT_ID = "your-project-id"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.genai import types
from pydantic import BaseModel
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google import adk

import warnings
import logging
from pprint import pprint
import json
import time

# Utils

In [ ]:
def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()

    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()

        try:
            agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))
        except AttributeError as e:
            print(f"Error parsing event content: {e}")
            continue

        if is_final_response:
            final_response = event.content.parts[0].text if event.content.parts else "No content available"
            elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
            print(f'>>> Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-" * 30)
        elif function_calls:
            print('+++ Function Calls:')
            for function_call in function_calls:
                print(f"Function Name: {function_call.name}, Args: {function_call.args}")
        elif function_responses:
            print('--- Function Responses:')
            for function_response in function_responses:
                response_details = function_response.response
                recommended_list = list(response_details.values()) if response_details else []
                print(f"Function Name: {function_response.name}")
                print(f"Function Results: {json.dumps(recommended_list)}")
        else:
            print('No function calls or responses available.')
            print(f"Agent Response: {agent_res}")

    elapsed_time_ms = round((time.time() - start_time) * 1000, 3)
    print(f"Total elapsed time: {elapsed_time_ms} ms")

# Concepts: AGENT


In [ ]:
AGENT_MODEL = "gemini-2.0-flash" # the model used by the agent

In [ ]:
# My first agent
root_agent = Agent(
    model=AGENT_MODEL,
    name="PhysicsPal",
    instruction="""
    Explain the physics concepts in plain English so a beginner can understand
        """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=1000,
    ),
    description="Agent that explains concepts from Physics",
)

In [ ]:
# Constant
APP_NAME = "physics_pal_app" # name of your agent app
USER_ID = "user12345"         # the user id that is interacting with the agent - used for a new conversation with the agent

### Setup Runner and Session Service

- To manage conversations and execute the agent, we need two more components:

**SessionService:** Responsible for managing conversation history and state for different users and sessions. The InMemorySessionService is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged. We'll explore state persistence more in Step 4.

**Runner:** The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the SessionService, and yields events representing the progress of the interaction.



In [ ]:
# define the session service
session_service = InMemorySessionService()

In [ ]:
# define runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

In [ ]:
#create session
session = await session_service.create_session(
     app_name=APP_NAME,
     user_id=USER_ID,
)

In [ ]:
user_input = types.Part(text="hi")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
user_input = types.Part(text="What is inertia?")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
user_input = types.Part(text="What is velocity?")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
user_input = types.Part(text="Do you have the list of all questions that I asked you in this session?")

events = runner.run(user_id=session.user_id, session_id=session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
session_tmp = await session_service.get_session(user_id=USER_ID, session_id=session.id, app_name=APP_NAME)
session_tmp

In [ ]:
session_tmp.events[1]

## Concept: Tools

In [ ]:
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

    # Mock weather data
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}


In [ ]:
# Our second agent and it can can use tools!

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather, adk.tools.preload_memory_tool.PreloadMemoryTool()], # Pass the function directly
)

In [ ]:
# Constant
WEATHER_APP_NAME = "weather_app" # name of your agent app
WEATHER_USER_ID = "user42"         # the user id that is interacting with the agent - used for a new conversation with the agent

In [ ]:
# define the session and memory service
weather_session_service = InMemorySessionService()
weather_memory_service = InMemoryMemoryService()

In [ ]:
# create the session and the runner
weather_session = await weather_session_service.create_session(
                                app_name=WEATHER_APP_NAME,
                                user_id=WEATHER_USER_ID
                            )
weather_runner = Runner(agent=weather_agent,
                        app_name=WEATHER_APP_NAME,
                        session_service=weather_session_service,
                        memory_service=weather_memory_service)



In [ ]:
user_input = types.Part(text="hi I am planning a trip to Europe")

events = weather_runner.run(user_id=weather_session.user_id, session_id=weather_session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
# Get the completed session
completed_session1 = await weather_runner.session_service.get_session(app_name=WEATHER_APP_NAME, user_id=WEATHER_USER_ID, session_id=weather_session.id)

# Add this session's content to the Memory Service
print("\n--- Adding Session 1 to Memory ---")
memory_service = await weather_memory_service.add_session_to_memory(completed_session1)
print("Session added to memory.")

In [ ]:
# create a second session for the user
weather_session_2 = await weather_session_service.create_session(
                                app_name=WEATHER_APP_NAME,
                                user_id=WEATHER_USER_ID
                            )

In [ ]:
input = 'Do you remember anything about our last conversation?' # @param {type:"string"}
user_input = types.Part(text=input)

events = weather_runner.run(user_id=weather_session_2.user_id,
                            session_id=weather_session_2.id,
                            new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
# Get the completed session
completed_session1 = await weather_runner.session_service.get_session(app_name=WEATHER_APP_NAME, user_id=WEATHER_USER_ID, session_id=weather_session.id)

# Add this session's content to the Memory Service
print("\n--- Adding Session 1 to Memory ---")
memory_service = await weather_memory_service.add_session_to_memory(completed_session1)
print("Session added to memory.")

In [ ]:
input = 'How is the weather in London' # @param {type:"string"}
user_input = types.Part(text=input)

events = weather_runner.run(user_id=weather_session.user_id, session_id=weather_session.id, new_message=types.Content(role='user', parts=[user_input]))
pprint_events(events)

In [ ]:
# Get the completed session
completed_session1 = await weather_runner.session_service.get_session(app_name=WEATHER_APP_NAME, user_id=WEATHER_USER_ID, session_id=weather_session.id)

# Add this session's content to the Memory Service
print("\n--- Adding Session 1 to Memory ---")
memory_service = await weather_memory_service.add_session_to_memory(completed_session1)
print("Session added to memory.")